<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Quilodrán
- Nombre de alumno 2: Katherine Rutte


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/quilo98/Labs_MDS`

# Importamos librerias útiles

In [101]:
!pip install -qq xgboost optuna

In [102]:
!pip install -r requirements.txt

Processing c:\home\conda\feedstock_root\build_artifacts\backcall_1592338393461\work (from -r requirements.txt (line 4))


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\home\\conda\\feedstock_root\\build_artifacts\\backcall_1592338393461\\work'



# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [103]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\quilo\AppData\Local\Temp\ipykernel_9888\3184305967.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [104]:
# Librerias necesarias 
import pandas as pd
import numpy as np
import pandas as pd
import datetime as dt
from IPython.display import HTML
import joblib
#Libreria para plotear
import matplotlib.pyplot as plt

import os
import seaborn as sns
from scipy import stats
import re
import itertools
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import shutil
import random
import optuna
import time

In [105]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

In [106]:
## 1
#Definimos X e y
X = df.drop(columns=["quantity"])
y =  df["quantity"]

#Dividimos los datos en conjuntos de entrenamiento (70%), validación (20%) y prueba (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

# Verificar las formas de los conjuntos
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_val:", X_val.shape)
print("Tamaño de X_test:", X_test.shape)


Tamaño de X_train: (5219, 11)
Tamaño de X_val: (1498, 11)
Tamaño de X_test: (739, 11)


In [107]:
## 2
# FunctionTransformer para extraer date

#Definimos una función para extraer día, mes y año en formato categorico
def extract_date(df):
    dataframe = df.copy()
    dataframe["date"] = pd.to_datetime(dataframe["date"])
    dataframe["day"] = dataframe["date"].dt.day.astype("category")
    dataframe["month"] = dataframe["date"].dt.month.astype("category")
    dataframe["year"] = dataframe["date"].dt.year.astype("category")
    dataframe.drop(columns=["date"], inplace=True)
    
    return dataframe

#Creamos un FunctionTransformer
date_transformer = FunctionTransformer(extract_date, validate=False)

#Aplicamos la transformación
df_transformed = date_transformer.transform(df)

df_transformed.head()

,id,city,lat,long,pop,shop,brand,container,capacity,price,quantity,day,month,year
0,0,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280,31,1,2012
1,1,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727,31,1,2012
2,2,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848,31,1,2012
3,3,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050,31,1,2012
4,4,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696,31,1,2012


In [108]:
# Paleta de colores de los Juegos Panamericanos Santiago 2023

# Define los colores en formato hexadecimal
colores = ["#0074FF", "#FE4F3A", "#FFD200", "#00CCCC"]

# Crea la paleta personalizada
stgo2023 = sns.color_palette(colores)

# Configura Seaborn para utilizar la paleta personalizada
sns.set_palette(stgo2023)

In [109]:
numeric_features = ["lat", "long", "pop", "price"]
categorical_features = ["city", "shop", "brand", "container", "capacity", "day", "month", "year"]

In [110]:
# Crear un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Escalar las variables numéricas
        ('cat', OneHotEncoder(sparse_output=False, drop='first'), categorical_features)  # Codificar en caliente las variables categóricas
    ]
)

## Aqui se supone que se transforma en un dataframe, si comentas esta linea si corre
preprocessor.set_output(transform='pandas')


ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity', 'day', 'month', 'year'])])

In [111]:
def remove_prefix(X, prefix):
    if isinstance(X, pd.DataFrame):
        return X.rename(lambda x: x[len(prefix):], axis=1)
    elif isinstance(X, np.ndarray):
        return X  # Opcionalmente, puedes realizar alguna operación en el array aquí si es necesario
    else:
        raise ValueError("Tipo de entrada no compatible. Se esperaba DataFrame o ndarray.")
    
remove_prefix_transformer = FunctionTransformer(remove_prefix, kw_args={'prefix': 'num__'}, validate=False)

In [112]:
# Creamos el pipeline
dummy_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),    
    ('remove_num_prefix', remove_prefix_transformer),
    ('regressor', DummyRegressor())
])



In [113]:
dummy_pipeline.fit(X_train, y_train)

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function extract_date at 0x0000020D93AA20D0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity',
                                                   'day', 'month', 'year'])])),
                ('remove_num_prefix',
                 FunctionTransformer(func=<function remove_prefix at 0x0000020D93768D30>,
                                     kw_args={'prefix': 'num__'})),
                ('regressor', DummyRegressor())])

In [114]:
# Predecir sobre el conjunto de validación
y_val_pred = dummy_pipeline.predict(X_val)

# Calcular la métrica mean_absolute_error
mae = mean_absolute_error(y_val, y_val_pred)
print(f'Mean Absolute Error en el conjunto de validación: {mae}')

Mean Absolute Error en el conjunto de validación: 13308.134750658153


In [115]:
xgb_regresor = XGBRegressor()

El mae no da un valor de 13.308 aproximadamente, esto en el contexto del negocio hacer referencia a que las ventas predichas de nuestro modelo en promedio difiere en 13.308 unidades con el valor real vendido

In [116]:
# Pipeline de xgb
xgb_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),    
    ('remove_num_prefix', remove_prefix_transformer),
    ('regressor', xgb_regresor)  
])


In [117]:
# Predecir sobre el conjunto de validación
xgb_pipeline.fit(X_train, y_train)

y_val_pred_xgb = xgb_pipeline.predict(X_val)

# Calcular la métrica mean_absolute_error
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f'Mean Absolute Error en el conjunto de validación con XGB: {mae_xgb}')

Mean Absolute Error en el conjunto de validación con XGB: 2500.7375647999415


El MAE utilizando XGNRegressor da un valor de 2.500, se redujo bastante en comparación con el modelo que utiliza DummyRegressor entregando un mejor rendimiento en cuanto a la predicción de la variable objetivo "Quantity"

In [118]:
joblib.dump(dummy_pipeline, 'dummy_model.pkl')
joblib.dump(xgb_pipeline, 'xgb_model.pkl')

['xgb_model.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [119]:
# Pipeline de xgb

xgb_regresor = XGBRegressor(monotone_constraints={'price': (-1)})
xgb_mono_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),    
    ('remove_num_prefix', remove_prefix_transformer),
    ('regressor', xgb_regresor)  
])

In [120]:
# Predecir sobre el conjunto de validación
xgb_mono_pipeline.fit(X_train, y_train)

y_val_pred_xgb = xgb_mono_pipeline.predict(X_val)

# Calcular la métrica mean_absolute_error
mae_xgb_mono = mean_absolute_error(y_val, y_val_pred_xgb)
print(f'Mean Absolute Error en el conjunto de validación con XGB forzando una relación monotona negativa entre precio y cantidad: {mae_xgb_mono}')

Mean Absolute Error en el conjunto de validación con XGB forzando una relación monotona negativa entre precio y cantidad: 2581.126497439294


In [121]:
joblib.dump(xgb_mono_pipeline, 'xgb_mono_model.pkl')

['xgb_mono_model.pkl']

En este caso el MAE aumenta en comparación con el modelo anterior, esto puede deberse a que esa relación no siempe se cumple, por lo que al forzar la relación monotona entre ambos valores pudo reducir la capacidad de generalización del modelo aumentando el error en la predicción.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Primero para entender los hiperparámetros debemos entender el modelo XGBRegresor. XGBoostRegressor es un algoritmo de Gradient Boosting para regresión que construye un modelo predictivo combinando múltiples árboles de decisión débiles. Inicia con una predicción simple y mejora iterativamente ajustando árboles adicionales a los errores del modelo. Utiliza regularización y optimización de hiperparámetros para prevenir el sobreajuste y mejorar la precisión en la predicción de valores numéricos, por último la función objetivo del algoritmo minimiza la pérdida y la complejidad del modelo simultáneamente. Ahora se explican los hiperparámetros y su importancia:
- learning_rate: Es la tasa de aprendizaje que controla la contribución de cada árbol al modelo. Valores más bajos hacen que el modelo aprenda de manera más conservadora y pueden mejorar la generalización, pero puede tardar mucho más en ajustarse a los datos de entrenamiento. Con valores muy altos el modelo puede sobreajustarse, perdiendo capacidad de generalización.
- n_estimators: Es el número de árboles que se van a construir. Un valor más alto puede mejorar la capacidad del modelo para capturar patrones complejos, pero también aumenta el riesgo de sobreajuste.
- max_depth: Es la profundidad máxima de cada árbol. Un valor más alto permite a los árboles capturar patrones más complejos, pero también puede llevar a un sobreajuste.
- max_leaves: Es el número máximo de nodos terminales (hojas) en cada árbol. Controla la complejidad del árbol y su capacidad para adaptarse a los datos.
- min_child_weight: Es la cantidad mínima total de peso (hessiano) necesaria en un nodo para dividirlo. Puede ayudar a controlar la sensibilidad a los outliers. 
- reg_alpha y reg_lambda: Son términos de regularización que controlan la penalización en la función de pérdida por términos de complejidad del modelo. Ayudan a prevenir el sobreajuste. 
- min_frequency: Para OneHotEncoder, es la frecuencia mínima necesaria para convertir una categoría en una columna one-hot. Valores más bajos pueden resultar en más columnas one-hot y pueden influir en la complejidad del modelo.

En general los rangos propuestos para la optimización hacen sentido y ayudan a probar distintas combinaciones de los parámetros en un amplio rango ayudando a encontrar la combinación óptima para un mejor funcionamiento del modelo.

In [122]:
# transformamos nuestros datos de X_train y X_val


def objective(trial, X_train_opt, X_val_opt, y_train_opt, y_val_opt):

    # Definir los hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    # Definir los hiperparámetros a optimizar para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Crear el preprocesador con los hiperparámetros sugeridos
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(min_frequency=encoder_params['min_frequency']), categorical_features)
        ]
    )

    # Crear el modelo XGBoost con los hiperparámetros sugeridos y el preprocesador
    model = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    # Iniciar el cronómetro para controlar el tiempo de entrenamiento
    start_time = time.time()

    # Entrenar el modelo en el conjunto de entrenamiento
    model.fit(X_train_opt, y_train_opt)

    # Detener el cronómetro y calcular el tiempo de entrenamiento
    training_time = time.time() - start_time

    # Calcular la métrica de evaluación en el conjunto de validación
    y_val_pred_opt = model.predict(X_val_opt)
    mae_opt = mean_absolute_error(y_val_opt, y_val_pred_opt)

    # Controlar el tiempo de entrenamiento para no exceder los 5 minutos
    if training_time > 300:
        raise optuna.TrialPruned()

    return mae_opt

# Configurar el muestreo de Optuna
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val), n_trials=100, timeout=300)

# Obtener los mejores hiperparámetros
best_params = study.best_params
print(f"Mejores hiperparámetros: {best_params}")

# Obtener el número de trials
num_trials = len(study.trials)
print(f"Número de trials: {num_trials}")

# Obtener el MAE del mejor modelo
best_mae = study.best_value
print(f"MAE del mejor modelo: {best_mae}")




Mejores hiperparámetros: {'learning_rate': 0.057833938586390896, 'n_estimators': 921, 'max_depth': 9, 'max_leaves': 81, 'min_child_weight': 3, 'reg_alpha': 0.02878051883207965, 'reg_lambda': 0.3284726947622543, 'min_frequency': 0.04733297119248123}
Número de trials: 100
MAE del mejor modelo: 1952.505746920373


In [123]:
## Creamos una función para guardar el modelo:

def build_optimal_pipeline(best_params, date_transformer):
    # Obtener los mejores hiperparámetros
    xgb_params = {
        'learning_rate': best_params['learning_rate'],
        'n_estimators': best_params['n_estimators'],
        'max_depth': best_params['max_depth'],
        'max_leaves': best_params['max_leaves'],
        'min_child_weight': best_params['min_child_weight'],
        'reg_alpha': best_params['reg_alpha'],
        'reg_lambda': best_params['reg_lambda']
    }

    encoder_params = {
        'min_frequency': best_params['min_frequency']
    }

    # Crear el preprocesador con los mejores hiperparámetros
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(min_frequency=encoder_params['min_frequency']), categorical_features)
        ]
    )

    # Crear el pipeline con los mejores hiperparámetros y el preprocesador
    optimal_pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    return optimal_pipeline


In [124]:
xgb_optuna_pipeline = build_optimal_pipeline(best_params, date_transformer)

In [125]:
xgb_optuna_pipeline.fit(X_train,y_train)

y_val_pred_xgb_opt = xgb_optuna_pipeline.predict(X_val)

# Calcular la métrica mean_absolute_error
mae_xgb_opt = mean_absolute_error(y_val, y_val_pred_xgb_opt)

print(mae_xgb_opt)

1952.505746920373


In [126]:
joblib.dump(xgb_optuna_pipeline, 'xgb_optuna_model.pkl')

['xgb_optuna_model.pkl']

Luego de optimizar los hiperparámetros se obtienen los siguientes resultados:

- Número de trials: 100
- learning_rate: 0.05783393858639089
- n_estimators: 921
- max_depth: 9
- max_leaves: 81
- min_child_weight: 3
- reg_alpha: 0.02878051883207965
- reg_lambda: 0.3284726947622543
- min_frequency: 0.04733297119248123
- MAE del mejor modelo: 1952.505746920373

El mae se reduce considerablemente de 2500 a 1952 aprox, esto se debe a la optimización de hiperparámetros, lo que logra que realizando los ajustes correspondientes al modelo este pueda obtener mejores resultados, ajustandose de mejor manera a la naturaleza de los datos.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

El Pruning hace refierencia a la eliminación selectiva de nodos y ramas del árbol de decisión durante su crecimiento. El Pruning ayuda a evitar el sobreajuste al remover secciones no necesarias del árbol que pueden haber capturado patrones específicos de los datos de entrenamiento pero que no son representativos de patrones generales. En XGBoost, se utiliza la técnica de pruning para mejorar la generalización del modelo, asegurando que cada nuevo nodo o rama agregada al árbol realmente contribuya a la mejora de la métrica objetivo.

In [127]:
from optuna.integration import XGBoostPruningCallback

def objective(trial, X_train_opt, X_val_opt, y_train_opt, y_val_opt, date_transformer):
    # Definir los hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'eval_metric': 'mae',  # Configuración de la métrica de evaluación
    }

    # Definir los hiperparámetros a optimizar para OneHotEncoder
    encoder_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Crear el preprocesador con los hiperparámetros sugeridos
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(min_frequency=encoder_params['min_frequency']), categorical_features)
        ]
    )

    # Aplicar transformaciones al conjunto de entrenamiento
    X_train_transformed = date_transformer.fit_transform(X_train_opt)
    X_train_transformed = preprocessor.fit_transform(X_train_transformed)
    
    # Aplicar transformaciones al conjunto de validación
    X_val_transformed = date_transformer.transform(X_val_opt)
    X_val_transformed = preprocessor.transform(X_val_transformed)

    # Crear el modelo XGBoost con los hiperparámetros sugeridos y el preprocesador
    model = XGBRegressor(**xgb_params, verbosity=0)

    # Agregar el callback de prunning
    pruning_callback = XGBoostPruningCallback(trial, "mae")
    
    # Iniciar el cronómetro para controlar el tiempo de entrenamiento
    start_time = time.time()

    # Entrenar el modelo en el conjunto de entrenamiento
    model.fit(
        X_train_transformed,
        y_train_opt,
        eval_set=[(X_val_transformed, y_val_opt)],
    )

    # Detener el cronómetro y calcular el tiempo de entrenamiento
    training_time = time.time() - start_time

    # Controlar el tiempo de entrenamiento para no exceder los 5 minutos
    if training_time > 300:
        raise optuna.TrialPruned()

    # Obtener la mejor métrica de validación (MAE)
    mae_opt = min(model.evals_result()['validation_0']['mae'])

    return mae_opt

# Configurar el muestreo de Optuna
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val, date_transformer), n_trials=100, timeout=300,show_progress_bar = True)

# Obtener los mejores hiperparámetros
best_params = study.best_params
print(f"Mejores hiperparámetros: {best_params}")

# Obtener el número de trials
num_trials = len(study.trials)
print(f"Número de trials: {num_trials}")

# Obtener el MAE del mejor modelo
best_mae = study.best_value
print(f"MAE del mejor modelo: {best_mae}")


  0%|          | 0/100 [00:00<?, ?it/s]

[0]	validation_0-mae:12990.00964
[1]	validation_0-mae:12693.71735
[2]	validation_0-mae:12415.98537
[3]	validation_0-mae:12151.95472
[4]	validation_0-mae:11904.89519
[5]	validation_0-mae:11674.05159
[6]	validation_0-mae:11454.49152
[7]	validation_0-mae:11247.97465
[8]	validation_0-mae:11056.58626
[9]	validation_0-mae:10879.26202
[10]	validation_0-mae:10717.63520
[11]	validation_0-mae:10562.52069
[12]	validation_0-mae:10414.24075
[13]	validation_0-mae:10279.84295
[14]	validation_0-mae:10149.54115
[15]	validation_0-mae:10028.36791
[16]	validation_0-mae:9913.11182
[17]	validation_0-mae:9809.18637
[18]	validation_0-mae:9711.49372
[19]	validation_0-mae:9615.78720
[20]	validation_0-mae:9527.45990
[21]	validation_0-mae:9445.37731
[22]	validation_0-mae:9367.99745
[23]	validation_0-mae:9295.42846
[24]	validation_0-mae:9224.93605
[25]	validation_0-mae:9160.61395
[26]	validation_0-mae:9101.99437
[27]	validation_0-mae:9045.00045
[28]	validation_0-mae:8991.33241
[29]	validation_0-mae:8941.36485
[30]

Best trial: 0. Best value: 8345.82:   1%|          | 1/100 [00:02<03:28,  2.10s/it, 2.10/300 seconds]

[0]	validation_0-mae:12901.36449
[1]	validation_0-mae:12542.60576
[2]	validation_0-mae:12189.05617
[3]	validation_0-mae:11890.12790
[4]	validation_0-mae:11598.33156
[5]	validation_0-mae:11309.14211
[6]	validation_0-mae:11059.19048
[7]	validation_0-mae:10846.12832
[8]	validation_0-mae:10625.67451
[9]	validation_0-mae:10421.69475
[10]	validation_0-mae:10257.38683
[11]	validation_0-mae:10074.20165
[12]	validation_0-mae:9901.38054
[13]	validation_0-mae:9730.89898
[14]	validation_0-mae:9576.74708
[15]	validation_0-mae:9453.07976
[16]	validation_0-mae:9301.68284
[17]	validation_0-mae:9187.60441
[18]	validation_0-mae:9063.01940
[19]	validation_0-mae:8947.91315
[20]	validation_0-mae:8820.61411
[21]	validation_0-mae:8734.98576
[22]	validation_0-mae:8616.71069
[23]	validation_0-mae:8506.66145
[24]	validation_0-mae:8420.38887
[25]	validation_0-mae:8334.86898
[26]	validation_0-mae:8268.88113
[27]	validation_0-mae:8184.83745
[28]	validation_0-mae:8104.99923
[29]	validation_0-mae:8023.01785
[30]	val

Best trial: 1. Best value: 5617.33:   2%|▏         | 2/100 [00:03<02:57,  1.81s/it, 3.71/300 seconds]

...

[0]	validation_0-mae:12701.16692
[1]	validation_0-mae:12156.41741
[2]	validation_0-mae:11651.05636
[3]	validation_0-mae:11179.16312
[4]	validation_0-mae:10730.98705
[5]	validation_0-mae:10325.58116
[6]	validation_0-mae:9952.34288
[7]	validation_0-mae:9602.19679
[8]	validation_0-mae:9301.39727
[9]	validation_0-mae:8995.36525
[10]	validation_0-mae:8705.58821
[11]	validation_0-mae:8438.86528
[12]	validation_0-mae:8181.99792
[13]	validation_0-mae:7935.46662
[14]	validation_0-mae:7702.72437
[15]	validation_0-mae:7481.66617
[16]	validation_0-mae:7271.88121
[17]	validation_0-mae:7093.55352
[18]	validation_0-mae:6895.91019
[19]	validation_0-mae:6708.88042
[20]	validation_0-mae:6532.69452
[21]	validation_0-mae:6381.22449
[22]	validation_0-mae:6203.02228
[23]	validation_0-mae:6064.35677
[24]	validation_0-mae:5932.64596
[25]	validation_0-mae:5780.62987
[26]	validation_0-mae:5656.00059
[27]	validation_0-mae:5530.65927
[28]	validation_0-mae:5405.53394
[29]	validation_0-mae:5294.14920
[30]	validatio

Best trial: 88. Best value: 1962.21:  99%|█████████▉| 99/100 [04:02<00:05,  5.02s/it, 242.90/300 seconds]

[0]	validation_0-mae:12698.73916
[1]	validation_0-mae:12147.96005
[2]	validation_0-mae:11638.50095
[3]	validation_0-mae:11170.96941
[4]	validation_0-mae:10752.66243
[5]	validation_0-mae:10360.03790
[6]	validation_0-mae:10014.98619
[7]	validation_0-mae:9701.37347
[8]	validation_0-mae:9397.13146
[9]	validation_0-mae:9114.88611
[10]	validation_0-mae:8874.04299
[11]	validation_0-mae:8638.80565
[12]	validation_0-mae:8425.73334
[13]	validation_0-mae:8217.98712
[14]	validation_0-mae:8026.26088
[15]	validation_0-mae:7848.98198
[16]	validation_0-mae:7673.14722
[17]	validation_0-mae:7514.65784
[18]	validation_0-mae:7371.61726
[19]	validation_0-mae:7237.03346
[20]	validation_0-mae:7106.16018
[21]	validation_0-mae:6991.85824
[22]	validation_0-mae:6891.56815
[23]	validation_0-mae:6794.04362
[24]	validation_0-mae:6705.99713
[25]	validation_0-mae:6631.40721
[26]	validation_0-mae:6557.28873
[27]	validation_0-mae:6486.78282
[28]	validation_0-mae:6418.92166
[29]	validation_0-mae:6367.21657
[30]	validati

Best trial: 88. Best value: 1962.21: 100%|██████████| 100/100 [04:06<00:00,  2.46s/it, 246.36/300 seconds]

Mejores hiperparámetros: {'learning_rate': 0.05834892631598031, 'n_estimators': 991, 'max_depth': 9, 'max_leaves': 60, 'min_child_weight': 5, 'reg_alpha': 0.2031895533422336, 'reg_lambda': 0.6593602366458704, 'min_frequency': 0.05576062855671267}
Número de trials: 100
MAE del mejor modelo: 1962.2093681875313


In [128]:
xgb_pruning_pipeline = build_optimal_pipeline(best_params, date_transformer)

In [129]:
xgb_pruning_pipeline.fit(X_train,y_train)

y_val_pred_xgb_pru = xgb_pruning_pipeline.predict(X_val)

# Calcular la métrica mean_absolute_error
mae_xgb_pru = mean_absolute_error(y_val, y_val_pred_xgb_pru)

print(mae_xgb_pru)

1962.2093681875313


In [130]:
joblib.dump(xgb_pruning_pipeline, 'xgb_pruning_model.pkl')

['xgb_pruning_model.pkl']

Mejores hiperparámetros: {'learning_rate': 0.05834892631598031, 'n_estimators': 991, 'max_depth': 9, 'max_leaves': 60, 'min_child_weight': 5, 'reg_alpha': 0.2031895533422336, 'reg_lambda': 0.6593602366458704, 'min_frequency': 0.05576062855671267}
Número de trials: 96
MAE del mejor modelo: 1962.2093681875313

Luego de optimizar los hiperparámetros se obtienen los siguientes resultados:

- Número de trials: 96
- learning_rate: 0.05834892631598031
- n_estimators: 991
- max_depth: 9
- max_leaves: 60
- min_child_weight: 5
- reg_alpha: 0.2031895533422336
- reg_lambda: 0.6593602366458704
- min_frequency: 0.05576062855671267
- MAE del mejor modelo: 1962.2093681875313

En este modelo en comparación con el anterior se observa un aumento en los valores de todos los parámetros, con un número de trials menor, en particular las alzas mas grande se observan en los hiperparámetros relacionados con la regularización: reg_alpha y reg_lambda, estos aumentan de 0.028 0.203 y de 0.32 a 0.65 espectivamente, aumentando la penalización del modelo al regularizar. El mae en este caso es 1962 levemente mayor que en el caso anterior, esto puede deberse a que por temas de tiempo (max 5 min) el número de trials es 96, por lo que quizás si se hubieran completado los 100 se obtendría un mae menor y obteniendo un modelo que se ajuste mejor a los datos en comparación con el modelo anterior.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [131]:
# Inserte su código acá
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

En el gráfico del historial de optimización se observa que luego del trial número 14 los resultados llegan a obtener resultados notablemente mejores, volviendo en los siguientes trial a acercarse a los valores que dieron esos resultados.

In [132]:
from optuna.visualization import plot_parallel_coordinate
plot_parallel_coordinate(study)

En el gráfico de coordenadas paralelas se observa como muchas veces se repiten los valores para parámetros en específicos, es decir una vez que se encuentran valores que mejoren el rendimiento estos se prueban con otras combinaciones de parámetros, en particular se observa como por ejemplo para el parámetro max_depth se tomaron la mayoría de veces valores de 10, 9 y 8 mientras que los otros valores fueron utilizados pocas veces, algunas excepciones a esto son los parámetros: n_stimators, learning_rate y reg_lambda, donde los valores fueron variando en un rango mas amplio.

In [133]:
from optuna.visualization import plot_param_importances
plot_param_importances(study)

El parámetro con mayor importancia en el modelo es por lejos min_frequency de OneHotEncoder, lo que nos da a entender que el procesamiento de los datos es hasta mas relevante que los hiperparámetros utilizados en el modelo, al cambiar la forma en que se codifica la data a utilizar es muy relevante en el desempeño que pueda tener un modelo en particular.

In [134]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [135]:
data = {
    'Modelo': ['Baseline', 'XGB', 'XGB_Mono', 'XGB_Optuna', 'XGB_Pruning'],
    'MAE': [13308.13, 2500.73, 2581.13, 1952.51, 1962.21]
}

# Crear DataFrame
resumen = pd.DataFrame(data)

# Mostrar DataFrame
display(resumen)

,Modelo,MAE
0,Baseline,13308.13
1,XGB,2500.73
2,XGB_Mono,2581.13
3,XGB_Optuna,1952.51
4,XGB_Pruning,1962.21


De la tabla se observa que los mejores modelos son XGB_Optuna y XGB_Pruning, esto debido a que en estos modelos se optimizaron los hiperparámetros, permitiendo así que el modelo se ajuste mejor a los datos y/o se tenga un mejor preprocesamiento antes de aplicar el modelo. Luego los modelos que siguen en cuanto a rendimiento según su MAE son el XGB_mono que es el modelo donde forzamos una relación entre precio y cantidad y el modelo de XGB con los valores predeterminados, por último el modelo con peor rendimiento fue el baseline donde utilizamos un DummyRegressor

In [136]:

loaded_model = joblib.load('xgb_optuna_model.pkl')

y_test_pred = loaded_model.predict(X_test)

# Calcular y mostrar el MAE
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f'MAE sobre el conjunto de prueba: {mae_test}')

MAE sobre el conjunto de prueba: 1996.1911984496576


Se obtiene un MAE un poco mayor para el conjunto de test que para el conjunto de validación, esto puede deberse a que el modelo optimizado con optuna que es el que se utiliza para obtener el MAE del conjunto de test fue optimizado guiandose por el MAE obtenido del conjunto de validación, por lo cual intenta disminuir el MAE de dicho conjunto, al utilizarlo en datos nuevos se pueden obtener variaciones en la métrica mencionada, aún así el modelo presenta una buena capacidad de generalización ya que la diferencia entre MAEs es bastante pequeña.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>